Notebook by:
* Lorenzo Pannacci 1948926
* Francesco Proietti 1873188
* Selin Topaloglu 2113300
* Santiago Vessi 1958879

## Startup

In [1]:
######################
# LIBRARIES DOWNLOAD #
######################

install_packages = False
if install_packages:
    %pip install beautifulsoup4 tqdm pandas numpy matplotlib nltk geopy folium earthpy sklearn

In [1]:
####################
# LIBRARIES IMPORT #
####################

import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import os
import csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import folium
from geopy.geocoders import Bing
import branca

import nltk
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import heapq
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from importlib import reload

In [2]:
########################
# LOCAL MODULES IMPORT #
########################

from functions.config import *
from functions import crawler
from functions import parser
from functions import engine
from functions import visualizing
from functions import algorithmic_question

In [7]:
crawler = reload(crawler)
parser = reload(parser)
engine = reload(engine)
visualizing = reload(visualizing)

## 1. Data collection

For this homework, there is no provided dataset. Instead, you have to build your own. Your search engine will run on text documents. So, here
we detail the procedure to follow for the data collection. We strongly suggest you work on different modules when implementing the required functions. For example, you may have a ```crawler.py``` module, a ```parser.py``` module, and a ```engine.py``` module: this is a good practice that improves readability in reporting and efficiency in deploying the code. Be careful; you are likely dealing with exceptions and other possible issues! 

### 1.1. Get the list of master's degree courses

We start with the list of courses to include in your corpus of documents. In particular, we focus on web scrapping the [MSc Degrees](https://www.findamasters.com/masters-degrees/msc-degrees/). Next, we want you to **collect the URL** associated with each site in the list from the previously collected list.
The list is long and split into many pages. Therefore, we ask you to retrieve only the URLs of the places listed in **the first 400 pages** (each page has 15 courses, so you will end up with 6000 unique master's degree URLs).

The output of this step is a `.txt` file whose single line corresponds to the master's URL.

---

Firstly we observe that we can nagivate trough the different pages using the link `https://www.findamasters.com/masters-degrees/msc-degrees/?PG=n` and changing `n` with the number of the desidered page. We also observe that without waiting between a page load and the other after 30 pages the website thinks we are a bot and don't let us in, to fix this we decided wait some seconds before we open a new page.

---

In [3]:
crawler.get_courses_links()

File already exist and is complete. Using the previous version.


### 1.2. Crawl master's degree pages

Once you get all the URLs in the first 400 pages of the list, you:

1. Download the HTML corresponding to each of the collected URLs.
2. After you collect a single page, immediately save its `HTML` in a file. In this way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
3. Organize the downloaded `HTML` pages into folders. Each folder will contain the `HTML` of the courses on page 1, page 2, ... of the list of master's programs.
   
__Tip__: Due to the large number of pages you should download, you can use some methods that can help you shorten the time. If you employed a particular process or approach, kindly describe it.

---

As before we have to insert a idle time between the loading of two pages to avoid that the website block us. We can found whether we have been blocked by checking if the webpage ha the title "`Just a moment...`". This makes the operations particularly slow, to crawl all pages we have to wait a few hours. To avoid to repeat this procedure more times than necessary we save the html pages and try to open them only if they are not already downloaded on the device.

---

In [4]:
crawler.crawl_pages()

All files already crawled. Using the existing version.


---

We observe that at the time of the crawl some pages as `https://www.findamasters.com/masters-degrees/course/emergency-management-and-resilience-msc/?i373d7361c25450` (page 215, course 3) were not available and gave us a filler webpage. We will have to treat those courses carefully as the only information we can get from those is the link. We can easily identify those kind of pages thanks to their title: "`FindAMasters | 500 Error : Internal Server Error`". The following function help us check if we correctly downloaded all the webpages.

---

In [ ]:
crawler.check_html_files()

### 1.3 Parse downloaded pages

At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format is as follows:

1. Course Name (to save as ```courseName```): string;
2. University (to save as ```universityName```): string;
3. Faculty (to save as ```facultyName```): string
4. Full or Part Time (to save as ```isItFullTime```): string;
5. Short Description (to save as ```description```): string;
6. Start Date (to save as ```startDate```): string;
7. Fees (to save as ```fees```): string;
8. Modality (to save as ```modality```):string;
9. Duration (to save as ```duration```):string;
10. City (to save as ```city```): string;
11. Country (to save as ```country```): string;
12. Presence or online modality (to save as ```administration```): string;
13. Link to the page (to save as ```url```): string.

<div style="overflow-x:auto;">
<table>
<thead>
  <tr>
    <th>index</th>
    <th>courseName</th>
    <th>universityName</th>
    <th>facultyName</th>
    <th>isItFullTime</th>
    <th>description</th>
    <th>startDate</th>
    <th>fees</th>
    <th>modality</th>
    <th>duration</th>
    <th>city</th>
    <th>country</th>
    <th>administration</th>
    <th>url</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>0</td>
    <td> Accounting and Finance - MSc</td>
    <td>University of Leeds</td>
    <td>Leeds University Business School</td>
    <td>Full time</td>
    <td>Businesses and governments rely on [...].</td>
    <td>September</td>
    <td>UK: £18,000 (Total) International: £34,750 (Total)</td>
    <td>MSc</td>
    <td>1 year full time</td>
    <td>Leeds</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/accounting-and-finance-msc/?i321d3232c3891">Link</a></td>
  </tr>
  <tr>
    <td>1</td>
    <td> Accounting, Accountability & Financial Management MSc</td>
    <td>King’s College London</td>
    <td>King’s Business School</td>
    <td>Full time</td>
    <td>Our Accounting, Accountability & Financial Management MSc course will provide [...].</td>
    <td>September</td>
    <td>Please see the university website for further information on fees for this course.</td>
    <td>MSc</td>
    <td>1 year FT</td>
    <td>London</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/accounting-accountability-and-financial-management-msc/?i132d7816c25522">Link</a></td>
  </tr>
  <tr>
    <td>2</td>
    <td> Accounting, Financial Management and Digital Business - MSc</td>
    <td>University of Reading</td>
    <td>Henley Business School</td>
    <td>Full time</td>
    <td>Embark on a professional accounting career [...].</td>
    <td>September</td>
    <td>Please see the university website for further information on fees for this course.</td>
    <td>MSc</td>
    <td>1 year full time</td>
    <td>Reading</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/accounting-financial-management-and-digital-business-msc/?i345d4286c351">Link</a></td>
  </tr>
  <tr>
    <td>3</td>
    <td> Addictions MSc</td>
    <td>King’s College London</td>
    <td>Institute of Psychiatry, Psychology and Neuroscience</td>
    <td>Full time</td>
    <td>Join us for an online session for prospective [...].</td>
    <td>September</td>
    <td>Please see the university website for further information on fees for this course.</td>
    <td>MSc</td>
    <td>One year FT</td>
    <td>London</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/addictions-msc/?i132d4318c27100">Link</a></td>
  </tr>
  <tr>
    <td>4</td>
    <td> Advanced Chemical Engineering - MSc</td>
    <td>University of Leeds</td>
    <td>School of Chemical and Process Engineering</td>
    <td>Full time</td>
    <td>The Advanced Chemical Engineering MSc at Leeds [...].</td>
    <td>September</td>
    <td>UK: £13,750 (Total) International: £31,000 (Total)</td>
    <td>MSc</td>
    <td>1 year full time</td>
    <td>Leeds</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
  </tr>
  <!-- Add more rows here as needed -->
</tbody>
</table>
</div>


For each master's degree, you create a `course_i.tsv` file of this structure:

```
courseName \t universityName \t  ... \t url
```

If an information is missing, you just leave it as an empty string.

---

We can observe that some informations are "mandatory", that means that every page (that is not a filler page) has them while others could or could not be present. Meanwhile filler pages gives us no information whatsoever, for those we have only the page url. To create a `.tsv` file we can just use the Python `csv` module changing its delimiter with the character `\t`. We use `separator = " "` for every text extraction as withou when a text is divided in more paragraphs when extracted the two strings are concatenated and this bring to merged words that would cause problems in later sections of the homework.

---

In [5]:
parser.html_extraction()

All files already created. Using the existing version.


## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the courses that match the query.

### 2.0 Preprocessing

#### 2.0.0) Preprocessing the text

First, you must pre-process all the information collected for each MSc by:

1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed
   
For this purpose, you can use the [`nltk library](https://www.nltk.org/).

---

The function takes a text string as input and returns a preprocessed version of the text. It utilizes regular expressions to remove all punctuation from the input text. After this punctuation removal, the text is tokenized by splitting it into words. Stopwords, common words that often don't contribute much to the meaning, are then removed from the tokenized text. The function employs stemming using the Porter Stemmer to reduce words to their root form, helping to consolidate similar words. Finally, the preprocessed words are joined back together into a single string, creating the final output.

---

In [10]:
# function 'preprocess_text' is in 'engine.py', we provide here an example

test_text = """
            This course will kick-start your professional career,
            providing you with advanced knowledge of modern financial theories, accounting practices and controls,
            as well as an understanding of the organisational context.
            """

engine.preprocess_text(test_text)

'thi cours kick start profession career provid advanc knowledg modern financi theori account practic control well understand organis context'

#### 2.0.1) Preprocessing the fees column

Moreover, we want the field ```fees``` to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a ```float``` column renamed ```fees (CHOSEN COMMON CURRENCY)```.

---

This function takes as input a string of text. It starts by finding the currency symbol. The text is previously transformed into its uppercase form to simplify the search. We then search for all possible fees and choose the highest one found. If both are found, we use the Open Exchange Rates API to convert the currency to Euros. Throughout the text, we might find various ways in which the value and currency appear. Sometimes, they will be properly separated by a space bar, while other times both parameters will be joined together. We will consider both of these cases. Also, the value might appear before or after the currency. Finally, for the dollar, pounds, and euros currencies, there are more ways in which they can appear. For this special case, we created a special dictionary that will help translate them into the correct form. So, when we apply the request function to the website, we will have the correct parameters for these currencies.

---

In [13]:
# function 'convert_to_eur' is in 'engine.py', we provide here an example
convert_text = "15.000 USD"

print(engine.convert_to_eur(convert_text), "EUR")

13759.26 EUR


---

When then create a function that explot the new `convert_to_eur` function and add to our `.tsv` files a new column that stores the value extracted as a number representing the fee of the course.

---

In [ ]:
engine.fees_preprocessing()

### 2.1. Conjunctive query

For the first version of the search engine, we narrowed our interest to the __description__ of each course. It means that you will evaluate queries only concerning the course's description.

#### 2.1.1) Create your index!

Before building the index, 
* Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

---

To create the vocabulary, we begin by initializing an empty dictionary. For each of the 6000 tsv files, we extract the description field. After preprocessing the text, we iterate through each word, checking if it's already in the vocabulary. If not, we add it and assign a unique ID. Finally, the vocabulary is saved in a txt file. We also created a method to load the vocabulary from a .txt file.

---

In [ ]:
engine.create_vocabulary()

In [5]:
vocabulary = engine.load_vocabulary(vocabulary_file_path)

Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

```
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```
where _document\_i_ is the *id* of a document that contains that specific word.

__Hint:__ Since you do not want to compute the inverted index every time you use the Search Engine, it is worth thinking about storing it in a separate file and loading it in memory when needed.

---
An empty dictionary, inverted_index, is created to store the inverted index, where each *term_ID* points to a list of documents containing that term. For each tsv file, we check if there are more than 4 fields to ensure the presence of a description field. The description field is then preprocessed using the preprocess_text function, and the resulting text is tokenized into words. For each word in the tokenized description:

- Check if the word contains alphabetical characters or numbers.
- If the word is in the vocabulary, update the inverted index:
- - Obtain the term ID for the word from the vocabulary.
- - Check if the term ID is already in the inverted index. If not, create a new entry with the tsv file.
- - If the term ID is already in the inverted index, add the document to the existing list, but only if it's not already present.

After processing all course descriptions, the inverted index is saved to a file (inv_index_file_path) with each line containing a term ID and the list of documents containing that term, separated by a tab.

---

In [4]:
engine.create_inverted_index(vocabulary)

  0%|          | 0/6000 [00:00<?, ?it/s]

Inverted index successfully created!


In [6]:
inverted_index = engine.load_inverted_index(inv_index_file_path)

#### 2.1.2) Execute the query

Given a query input by the user, for example:

```
advanced knowledge
```

The Search Engine is supposed to return a list of documents.

**What documents do we want?**

Since we are dealing with conjunctive queries (AND), each returned document should contain all the words in the query.
The final output of the query must return, if present, the following information for each of the selected documents:

* `courseName`
* `universityName`
* `description`
* `URL`

If everything works well in this step, you can go to the next point and make your Search Engine more complex and better at answering queries.

---
The input query is preprocessed using the preprocess_text function, and the resulting text is tokenized into individual words. The variable *doc* is initialized as an empty set. It will eventually contain all the documents that have the complete query in their description.
The code iterates through each word in the processed query:
- For each word, it checks if the word is in the vocabulary. If so, it retrieves the term ID.
- If the term ID is in the inverted index, it collects the set of documents associated with that term ID.
- If it's the first word in the query, it updates the doc set. Otherwise, it updates a temporary set tmp.
- It then filters out the documents that don't contain the previous word of the query (for words beyond the first).

For each document ID in the final doc set, it reads the corresponding tsv file.
Relevant information from the file, such as course name, university name, description, and URL, is extracted and stored in a list of dictionaries (*result_data*).
The extracted information is used to create a Pandas DataFrame (result_df), which is then returned by the function.


---

In [16]:
# function 'search_engine' is in 'engine.py', we provide here an example

query = "advanced knowledge"
engine.search_engine(query, vocabulary, inverted_index)

,courseName,universityName,description,URL
0,Engineering Management MSc,University of Bradford,The MSc Engineering Management programme will ...,https://www.findamasters.com/masters-degrees/c...
1,Building Information Modelling and Project Col...,University of Derby,Aligned with the current and future needs of t...,https://www.findamasters.com/masters-degrees/c...
2,History of International Relations MSc,London School of Economics and Political Science,Ask LSE The MSc History of International Relat...,https://www.findamasters.com/masters-degrees/c...
3,"Information Systems, Master Programme (60 cred...",Linnaeus University,Information systems offer opportunities for en...,https://www.findamasters.com/masters-degrees/c...
4,Food and Nutrition Sciences (MSc),Sheffield Hallam University,Gain advanced food industry knowledge and prac...,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...
479,Cancer Biology and Precision Oncology (Online ...,University of Edinburgh,Programme description This new programme will ...,https://www.findamasters.com/masters-degrees/c...
480,Construction Engineering Management MSc,University of East London,The programme is designed to meet the demand w...,https://www.findamasters.com/masters-degrees/c...
481,Advanced Computer Science (Data Analytics) - MSc,University of Leeds,"From science to marketing, engineering to medi...",https://www.findamasters.com/masters-degrees/c...
482,Data Science with Artificial Intelligence MSc,University of Exeter,Overview Designed for those interested in lear...,https://www.findamasters.com/masters-degrees/c...


### 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

- Find all the documents that contain all the words in the query.
- Sort them by their similarity with the query.
- Return in output *k* documents, or all the documents with non-zero similarity with the query when the results are less than *k*. You must use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

To solve this task, you must use the *tfIdf* score and the *Cosine similarity*. The field to consider is still the description. Let's see how.

#### 2.2.1) Inverted index

Your second Inverted Index must be of this format:

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}
```

Practically, for each word, you want the list of documents in which it is contained and the relative tfIdf score.

Tip: TfIdf values are invariant for the query. Due to this reason, you can precalculate and store them accordingly.

---
The code initializes an inverted index *inv_index_tfid* and dictionaries for term frequency *t_f* and document frequency *d_f*.
The code iterates through a range of document IDs and reads the corresponding TSV files. 
Term Frequency Calculation:
- The code iterates through each word in the list of words extracted from the document description (preprocessed).
- It retrieves the term ID for each word from the vocabulary.
- If the term ID exists it updates the term frequency dictionary (t_f) for the pair (term_id, tsv) by incrementing the count.

Document Frequency (DF) Calculation:
- The code initializes a set to keep track of unique terms encountered in the document.
- It then iterates through the set of unique words in the document.
- For each word, it retrieves the term ID from the vocabulary.
- If the term ID exists, and it hasn't been seen before in the document, it updates the document frequency dictionary for the term ID by incrementing the count. It also adds the term ID to the set of seen words.

Then the TF-IDF score is calculated and rounded to two decimal places. Finally it updates, the inverted index with the term ID and a tuple containg the ID and it's TF-IDF score.
 
---

In [7]:
engine.create_inverted_index_tfidf(vocabulary)

  0%|          | 0/6000 [00:00<?, ?it/s]

Inverted index TF-IDF successfully created!


In [9]:
inverted_index_tfidf = engine.load_inverted_index(inv_ind_tfidf_file_path)

#### 2.2.2) Execute the query

In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the tfIdf representations of the documents.

Given a query input by the user, for example:

```
advanced knowledge
```

The search engine is supposed to return a list of documents, ranked by their Cosine Similarity to the query entered in the input.

More precisely, the output must contain:

- courseName
- universityName
- description
- URL
- The similarity score of the documents with respect to the query (float value between 0 and 1)

---

The function starts by preprocessing the query and vectorizing it using the TfidfVectorizer function, calculating the TF-IDF score for each term. Following this, it identifies documents that contain all the query words in their description, leveraging the same method as the search_engine function. Afterward, the function computes the cosine similarity for each document. This is achieved using the query vector and the document vector, where the document vector is constructed from the TF-IDF scores in the inverted index file. The function employs a heap data structure to efficiently maintain the top documents based on their cosine similarity scores. In cases where there are fewer than k documents satisfying the query conditions, the function returns all available documents.

---

In [25]:
# function 'search_engine' is in 'engine.py', we provide here an example

query = "advanced knowledge"
engine.top_k_documents(query, vocabulary, inverted_index, inverted_index_tfidf)

,courseName,universityName,description,URL,similarityScore
0,Advanced Computing MSc,King’s College London,Our Advanced Computing MSc provides knowledge ...,https://www.findamasters.com/masters-degrees/c...,0.324157
1,Advanced Healthcare Practice - MSc,Cardiff University,Why study this course Our MSc Advanced Healthc...,https://www.findamasters.com/masters-degrees/c...,0.313722
2,Advanced Clinical Practice MSc,University of Greenwich,Learn essential strategies and prepare for lea...,https://www.findamasters.com/masters-degrees/c...,0.309663
3,Advancing Practice - MSc,University of Northampton,Our MSc Advancing Practice awards support the ...,https://www.findamasters.com/masters-degrees/c...,0.304063
4,Advanced Clinical Practice - MSc,Canterbury Christ Church University,Gain the knowledge and skills needed to become...,https://www.findamasters.com/masters-degrees/c...,0.302410


## 3. Define a new score!

Now it's your turn: build a new metric to rank MSc degrees.

Practically:

1. The user will enter a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 2.1.
2. Once you have the documents, you need to sort them according to your new score. In this step, you won't have any more to take into account just the ```description``` field of the documents; you __can__ use also the remaining variables in your dataset (or new possible variables that you can create from the existing ones or scrape again from the original web-pages). You __must__ use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

__N.B.:__ You have to define a __scoring function__, not a filter! 

The output, must contain:

* `courseName`
* `universityName`
* `description`
* `URL`
* The  __new__ similarity score of the documents with respect to the query

Are the results you obtain better than with the previous scoring function? **Explain and compare results**.

In [174]:
def scoring_function(query, result_df, k = 5, all_rows = False):

    # score based on the presence of the full query in that spesific order in the description
    result_df['score_description'] = result_df['description'].apply(lambda x: query in x).astype(int)

    # score based on the presence of any part of the query in the course name
    result_df['score_course_name'] = result_df['courseName'].apply(lambda x: any(word in x for word in query.split())).astype(int)

    # score based on the presence of empty rows
    result_df['score_empty_info'] = result_df.apply(lambda row: -1 if row.isnull().all() else 0, axis=1)

    # score based on the proximity of the start date to the current month
    result_df['score_start_date'] = result_df.apply(lambda row: calculate_start_date_score(row['startDate']), axis=1)

    # combination of the scores with weights based on importance
    result_df['total_score'] = (
        0.4 * result_df['score_description'] +
        0.3 * result_df['score_course_name'] +
        0.1 * result_df['score_empty_info'] +
        0.2 * result_df['score_start_date']
    )

    # sort the DataFrame based on the total score
    result_df = result_df.sort_values(by='total_score', ascending=False)

    # TO REVISE
    # IT DOESEN'T USE HEAP CORRECTLY

    # use a heap data structure to maintain the top-k documents
    top_k = heapq.nlargest(k, zip(result_df['courseName'], result_df['universityName'], result_df['description'], result_df['url'], result_df['total_score']))

    # return the top-k documents
    if all_rows: # TO REVISE, JUST INSERTED TO RETURN FEES FOR POINT 4
        top_k = heapq.nlargest(k, zip(result_df['courseName'], result_df['universityName'], result_df['city'], result_df['country'], result_df['description'], result_df['fees (EUR)'], result_df['url'], result_df['total_score']))

        return pd.DataFrame(top_k, columns=['courseName', 'universityName', 'city', 'country', 'description', 'fees (EUR)', 'url', 'total_score'])
    else:
        top_k = heapq.nlargest(k, zip(result_df['courseName'], result_df['universityName'], result_df['description'], result_df['url'], result_df['total_score']))

        return pd.DataFrame(top_k, columns=['courseName', 'universityName', 'description', 'url', 'total_score'])

def calculate_start_date_score(start_date):
    # positive score is given if the start date is closer to the current month
  
    current_month = pd.to_datetime('today').month
    start_month = parse_start_date(start_date)
    # calculate the difference, considering circular months
    months_difference = (start_month - current_month) % 12  

    # positive score if the start date is within 3 months
    if months_difference <= 6:  
        return 1
    else:
        return 0

def parse_start_date(startDate):
    # extract the current month from the start date
    
    try:
        return pd.to_datetime(startDate, format='%B').month
    except ValueError:
        return -1  # return -1 for invalid dates such as "See Course"

In [175]:
query = "advanced knowledge"
result_df = search_engine(query, all_rows = True)
top_k_results = scoring_function(query, result_df)
top_k_results

,courseName,universityName,description,url,total_score
0,Stroke Medicine MSc,Learna | Diploma MSc,Postgraduate training in stroke care is limite...,https://www.findamasters.com/masters-degrees/c...,0.2
1,Physics - MSc,University of Leeds,Studying the Physics MSc at Leeds will give yo...,https://www.findamasters.com/masters-degrees/c...,0.0
2,Materials Science and Engineering - MSc,University of Leeds,Materials science is at the forefront of provi...,https://www.findamasters.com/masters-degrees/c...,0.0
3,Masters in Hospitality Management,Ecole hotelière de Lausanne,"With our Master in Hospitality Management, you...",https://www.findamasters.com/masters-degrees/c...,0.0
4,Masters in Economics,University of Lisbon,OBJECTIVES The MSc in Economics aims to provid...,https://www.findamasters.com/masters-degrees/c...,0.4


## 4. Visualizing the most relevant MSc degrees

Using maps can help people understand how far one university is from another so they can plan their academic careers more adequately. Here, we challenge you to show a map of the courses found with the score defined in point 3. You should be able to identify at least the *city* and *country* for each MSc degree. You can find some ideas on how to create maps in Python [here](https://plotly.com/python/maps/) and [here](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621) but you will maybe need further information for a proper visualization, like coordinates (latitude and longitude). You can retrieve this data using various tools:

1. [Here](https://medium.com/@manilwagle/geocoding-the-world-using-google-api-and-python-1f6b6fb6ca48) you can find a helpful tutorial on how to encode geo-informations using Google API in Python (this tool can also be used in [Google Sheets](https://handsondataviz.org/geocode.html))
2. You can collect a list of unique places in the format (City, Country) and ask chatGPT (or, as usual, any other LLM chatbot) to provide you with a list of corresponding representative coordinates
3. Explore and find the best solution for your case!
   
Once you defined your visualization strategy, include a way to encode fees in your charts. The map should show (with a proper legend) different courses and associated taxation: the user wants a glimpse not only of how far he will need to move but also of how much it will cost him!

---

Given the courses obtained from the previous points, we have for every entry in our courses dataset to first convert `city` and `country` and `universityName` into latitude and longitude. For doing that we use the library `geopy` and as geocoder `Bing`, for which is required an API key that is here omitted. To avoid useless repetitions in the api calls we save the conversion from address to coordinates in a `.csv` file. We create a map from every university in our dataset to their coordinates, as we don't previously know which courses will be selected in the query executed in point 3.

---

In [5]:
coordinates_df = visualizing.get_coordinates_table("AtdaJRct2rWxYPJrhVFVVqjLV39daVFysafTBx5zoAaa6cmWcNULVi-jywbYVIDB")
coordinates_df

Table already created!


,universityName,city,country,latitude,longitude
0,AFG College with the University of Aberdeen,Doha,Qatar,25.285633,51.526417
1,Aalborg University,Aalborg,Denmark,57.015285,9.978995
2,Aalborg University Copenhagen Campus,Copenhagen,Denmark,55.698200,12.571623
3,Aalto University,Tampere,Finland,61.498020,23.760311
4,Aberdeen University,Aberdeen,United Kingdom,57.164803,-2.102050
...,...,...,...,...,...
330,Xi’an Jiaotong-Liverpool University,Suzhou,China,31.274799,120.738068
331,York St John University,York,United Kingdom,53.965401,-1.080673
332,York University (Canada),Toronto,Canada,43.773705,-79.503281
333,Zhejiang Gongshan University,Hangzhou,China,30.245920,120.210793


---

After this conversion we create an interactive map using the library `folium` and populate it with markers for every relevant course.

**WIP: Real data missing, for now we use all the entries in our dataset.**

* make marker change color following the cost
* make legend
* upload coordinates converter on aws

---

In [8]:
visualizing.visualize_map("advanced knowledge", coordinates_df)

  0%|          | 0/6000 [00:00<?, ?it/s]

## 5. BONUS: More complex search engine

__IMPORTANT:__ This is a bonus step, so it's <ins>not mandatory</ins>. You can get the maximum score also without doing this. We will take this into account, **only if** the rest of the homework has been completed.

For the Bonus part, we want to ask you more sophisticated search engine. Here we want to let users issue more complex queries. The options of this new search engine are: 
1. Give the possibility to specify queries for the following features (the user should have the option to issue __none or all of them__): 
* `courseName`
* `universityName`
* `universityCity`
2. Specify a range for the __fees__ to retrieve only MSc whose taxation is in that range.
3. Specify a list of __countries__ which the search engine should only return the courses taking place in city within those countries.
4. Filter based on the courses that have already started. 
5. Filter based on the presence of online modality. 

__Note 1__: You should be aware that you should give the user the possibility <ins>to select any</ins> of the abovementioned options. How should the user use the options? We will accept __any manual__ that you provide to the user. 

__Note 2__: As you may have realized from __1st option__, you need to build <ins>inverted indexes</ins> for those values and return all of the documents that have the similarity <ins>more than 0</ins> concerning the given queries. Choose a __logical__ way to aggregate the similarity coming from each of them and explain your idea in detail.

__Note 3__: The options <ins>other than 1st</ins> one can be considered as __filtering criteria__ so the retrieved documents <ins>must respect all</ins> of those filters. 

The output must contain the following information about the places:

* `courseName`
* `universityName`
* `url`

In [5]:
# TODO

## 6. Command Line Question

As done in the previous assignment, we encourage using the command as a feature that Data Scientists must master.

Note: To answer the question in this section, you must strictly use command line tools. We will reject any other method of response. The final script must be placed in CommandLine.sh.

First, take the course_i.tsv files you created in point 1 and merge them using Linux commands (Hint: make sure that the first row containing the column names appears only once).

Now that you have your merged file named merged_courses.tsv, use Linux commands to answer the following questions:
- Which country offers the most Master's Degrees? Which city?
- How many colleges offer Part-Time education?
- Print the percentage of courses in Engineering (the word "Engineer" is contained in the course's name).

__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.

The next cell contains the 'CommandLine.sh' script:

```

#!/bin/bash


#command useful in order to format the output
paint=$(tput rev)
no_paint=$(tput sgr 0)
blue=$(tput setaf 4)
red=$(tput setaf 1)
green=$(tput setaf 2)
yellow=$(tput setaf 3)

#printing formatted title and introduction
echo -e "\n"
echo "$paint$red                      COMMAND LINE QUESTION HW3 AMDM                            $no_paint"
echo "$paint$red  $no_paint                                                                            $paint$red  $no_paint"
echo "$paint$red  $no_paint This bash script merges all the 6000 files .tsv in one and answers to      $paint$red  $no_paint"
echo "$paint$red  $no_paint the three questions by analysing the .tsv file created.                    $paint$red  $no_paint"
echo "$paint$red  $no_paint                                                                            $paint$red  $no_paint"
echo "$paint$red                                                                                $no_paint"
echo -e "\n"
echo "Please wait a few seconds, untill you see the result on standard output, the machine is calculating..."
echo "For a clearly visualization of the output it's recommended to maximize the terminal window..."
echo -e "\n"


#################
#               #
# Merging files #---------------------------------------------------
#               #
#################

#inizialization of the merged_file with the headers
head -n1 course_1.tsv > merged_courses.tsv

#appending rows to the merged_file
for file in course*.tsv
do
    tail -n1 $file >> merged_courses.tsv
done


#################################
#                               #
# Which country and which city? #-----------------------------------
#                               #
#################################

#assegnation of variable useful for calculate the max
max_1=0
max_country=' '

#extracting the countries column 
cut -f11 merged_courses.tsv | sed 1d | sort -u > countries.tsv

#this command says to the for loop to consider the entire line as a variable 
IFS=$'\n'

#for loop along all the countries
for country in $(cat 'countries.tsv')
do
    #l contains the occurrence of the country
    l=$(cut -f11 merged_courses.tsv | grep -i $country | wc -l)
    
    #if statement in order to compare and extract the max
    if [ $l -ge $max_1 ]
    then
	max_1=$l
	max_country=$country
    fi
done

#this part works as the previous
max_2=0
max_city=' '
cut -f10 merged_courses.tsv | sed 1d | sort -u > cities.tsv

IFS=$'\n'
for city in $(cat 'cities.tsv')
do
    c=$(cut -f10 merged_courses.tsv | grep -i $city | wc -l)
    if [ $c -ge $max_2 ]
    then
	max_2=$c
	max_city=$city
    fi
done


####################
#                   #
# Part-time courses #-----------------------------------------------
#                   #
#####################

#extracting the columns of the university name and the time type
cut -f2,4 merged_courses.tsv | sed 1d | grep -i 'part time' | cut -f1 > univ_p-t.tsv

#sorting and deleting the duplicates we can calculate
#the number of university that offers part-time courses
num_univ=$(sort -u univ_p-t.tsv | wc -l)


##########################################
#                                        #
# Calculating the courses in Engineering #--------------------------
#                                        #
##########################################

#extracting the column about the courses' name
cut -f1 merged_courses.tsv | sed 1d > courseName.tsv

#counting the courses with 'Engineer' in their name
x=$(grep -i "Engineer" courseName.tsv | wc -l)

#counting the total courses (not counting the empty lines)
y=$(grep -vc '^$' courseName.tsv)

#calculating the percentage
z=$(echo "scale=3;$x*100.0/$y" | bc)


#removing the temporary files used for the analysis 
rm cities.tsv
rm countries.tsv
rm univ_p-t.tsv
rm courseName.tsv

#printing formatted output question 1
echo "$paint$blue    QUESTION 1: WHICH COUNTRY OFFERS THE MOST MASTER'S DEGREES? WHICH CITY?     $no_paint"
echo "$paint$blue  $no_paint                                                                            $paint$blue  $no_paint"
echo "$paint$blue  $no_paint The country that offers the greater number of Master's Degrees is:         $paint$blue  $no_paint"
echo "$paint$blue  $no_paint "$max_country" with "$max_1" courses.                                          $paint$blue  $no_paint"
echo "$paint$blue  $no_paint The city that offers the greater number of Master's Degree is: "$max_city"      $paint$blue  $no_paint"
echo "$paint$blue  $no_paint with "$max_2" courses                                                          $paint$blue  $no_paint"
echo "$paint$blue  $no_paint                                                                            $paint$blue  $no_paint"
echo "$paint$blue                                                                                $no_paint"

#question 2
echo "$paint$green    QUESTION 2: HOW MANY COLLEGES OFFER PART-TIME EDUCATION?                    $no_paint"
echo "$paint$green  $no_paint                                                                            $paint$green  $no_paint"
echo "$paint$green  $no_paint The number of colleges that offer part-time education is: "$num_univ"              $paint$green  $no_paint"
echo "$paint$green  $no_paint                                                                            $paint$green  $no_paint"
echo "$paint$green                                                                                $no_paint"

#question 3
echo "$paint$yellow    QUESTION 3: PRINT THE PERCENTAGE OF COURSES IN ENGINEERING                  $no_paint"
echo "$paint$yellow  $no_paint                                                                            $paint$yellow  $no_paint"
echo "$paint$yellow  $no_paint The percentage of courses in engineering is: "$z"%                       $paint$yellow  $no_paint"
echo "$paint$yellow  $no_paint                                                                            $paint$yellow  $no_paint"
echo "$paint$yellow                                                                                $no_paint"

echo -e "\n"

```

The screenshot below contains the output of the bash script runned on local PC command line using Ubuntu Linux:

![output_screenshot](CLQ_screen.png)

## 7. Algorithmic Question 

Leonardo is an intern at a company. He is paid based on the total number of hours he has worked. They agreed __d__ days ago that Leonardo could not work less than $minTime_i$ or more than $maxTime_i$ hours per <ins>i-th</ins> day. Furthermore, he was warned by HR that on his last day at the company, he should provide a detailed report on how many hours he worked <ins>each day</ins> for the previous d days.

Today is the day Leonardo should report to HR, but the problem is that he <ins>didn't</ins> account for how many hours he put in for each day, so he only has the __total sum of the hours__ ($sumHours$) he put in total in these d days. He believes that if he creates a report in which each number $dayHours_i$ corresponds to the __total hours he worked on the i-th day__ while satisfying the HR limitations and the total sum of all $dayHours_i$ equals $sumHours$, he would be fine.

He cannot create such a report independently and requests your assistance. He will give you the number of days $d$, total hours spent $sumHours$, and the HR limitations for each day $i$, and he wants you to assist him in determining whether it is possible to create such a fake report. If that is possible, make such a report. 

**Input**

The first line of input contains two integers __d__, $sumHours$ - the number of days Leonardo worked there and the total number of hours he worked for the company. Each of the following __d__ lines contains two integer numbers $minTime_i$ and $maxTime_i$ - the minimum and maximum hours he can work on the $i_{th}$ day. 

**Output**

If such a report cannot be generated, print 'NO' in one output line. If such a report is possible, print 'YES' in the output and d numbers - the number of hours Leonardo spent each day - in the second line. If more than one solution exists, print any of them. 

__Input 1__
```
2 5
0 1
3 5
```
__Output 1__
```
YES
1 4 
```
---
__Input 2__
```
1 1
5 6
```
__Output 2__
```
NO
```

---

**7.1 Implement a code to solve the above mentioned problem.**

In [29]:
# the algorithm is in 'algorithmic_question.py', here we show the output for the two presented examples
 
input1 = "2 5\n0 1\n3 5"
print("Test1:")
algorithmic_question.algorithm(input1)


input2 = "1 1\n5 6"
print("\nTest2:")
algorithmic_question.algorithm(input2)

Test1:
YES
1 4

Test2:
NO


**7.2 What is the __time complexity__ (the Big O notation) of your solution? Please provide a <ins>detailed explanation</ins> of how you calculated the time complexity.**

---

We can see the algorithm as divided in two parts: the first one calculate if it's possible to satisfy the condition and the second one create a possible distribution of the hours troughtout the days. Calling `d` the number of days and therefore the number of lines of the input the first part has a cycle of doing the following operations:

* Reading and splitting the input (costant)
* Checking if costraint is impossible (constant)
* Append values to pre-allocated list (costant)
* Sum values in accumulator (costant)

And after that a check on the total costraints of the problem, that is costant too. The worst case scenario is when there is no impossible costraint, this means that every line is read and this brings the cost of the first part to `O(d)`. The second part creates the output of the function. If the costraints obtained in the first part were unsatisfiable we just print "NO" (costant time). If the problem is satisfiable we have to create a possible list of hours that satisfy all the costraints: for every day it has to be $\ge$ than then minimum, $\le$ than the maximum of that specific day and the sum has to be exactly `sumHours`.

To reach the last costraint we decided to start with all the days setted to the minimum hour and then increase every day to the max until that would surpass `sumHours`, at which we just set the hours at the right amount such that the sum becomes `sumHours`. In the setup of the cycle we have some costant time operations and the creation of the `diffs` array, that contains all the differences between max and min for every day, creating that array has cost `O(d)`. Then have to cycle trough all the days and do this operation. Using numpy arrays the array indexing become costant, so an iteration of a cycle uses costant time. The worst-case scenario is when we have to cycle trough all the days (EX. `sumHours` is equal to the sum of all the `max`), and this brings the cost of the second part to `O(d)`.

Since both parts are linear we can say that the cost of the algorithm is linear `O(d)`, with `d` the size of the input.

**7.3 Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the <ins>two differ</ins>, which one is right? (why?)**

---

We asked ChatGPT 3.5 to evaluate the time complexity of the code using the following prompt:

```
You are asked to evaluate the following algorithm wirtten in python and evaluate its time complexity in Big O notation in the worst-case scenario.
[python code of the algorithm]
```

And we obtained the following response:

```
This algorithm seems to involve iterating through each constraint provided in the input and checking certain conditions. Let's break down the time complexity:

Getting the first line of input and initializing variables takes constant time, denoted as O(1).
Iterating through the input constraints involves going through 'd' iterations, where 'd' represents the number of days. Within each iteration:
Splitting the input and mapping values takes constant time, O(1).
Checking if min > max and updating lists involves constant time operations, O(1).
Adding values to the lists and updating sums takes constant time, O(1).
Checking the total constraints satisfaction also takes constant time, O(1).
The nested loop and operations inside it iterate through 'd' constraints, making the time complexity for this part O(d) in total.

The last part of the algorithm involves another loop through 'd' constraints to determine the output. Within this loop:

The majority of operations inside the loop are constant time operations, O(1).
However, the loop might break prematurely, depending on the conditions, leading to an average case scenario of approximately d/2 iterations.
The overall time complexity of this algorithm in the worst-case scenario is approximately O(d) considering the dominant part is the iteration through 'd' constraints.
```

The response of ChatGPT is very similar to ours, both on the results and (surprisingly) in the form. The LLM seems to have analyzed the code dividing it in two parts as we did. We want to highlight that it decided to talk about "average case scenario" even if we explicitely said to consider the "worst-case scenario": in this specific case it has brought no significant effect, as $d/2 \in O(d)$, however in the analysis of other algorithms that could have brought a different results EX. quicksort has a worst-case scenario of $O(n^2)$ while its average case scenario is $O(nlogn)$

**7.4 What do you think of the __optimality__ of your code? Do you believe it is optimal? Can you improve? Please <ins>elaborate</ins> on your response.**

---

We believe the code we have written is optimal. Since only reading the input once has a time complexity of $O(d)$ if an algorithm  has a time complexity lower than that means that the algorithm must not read all the input, however without reading all the input is impossible to know whether a certain number of hours set in the output for a certain day breaks some costraints.

Maybe in the average case scenario a little boost in the performance can be obtained in the second part if instead of calculating all the `diffs` at the same time we calculate them only when needed, as we probably don't need the latter ones, but even this could be debatable as the vectorization of the operation makes it very efficient.